# conversaciones

In [1]:
!pip install --upgrade spacy
!python -m spacy download es_core_news_sm
import spacy

# Cargar el modelo de idioma español de SpaCy
nlp = spacy.load('es_core_news_sm')

import spacy
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize.toktok import ToktokTokenizer
import re
import unicodedata

# Inicializar el tokenizador
tokenizer = ToktokTokenizer()

# Descargar la lista de stopwords en español de NLTK
nltk.download('stopwords')
stopword_list = nltk.corpus.stopwords.words('spanish')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 2.9 MB/s eta 0:00:0000:0100:01


✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


[nltk_data] Downloading package stopwords to /Users/sil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import pandas as pd
import numpy as np
from textblob import TextBlob

In [3]:
df = pd.read_csv("../data/data_capi_chat_clear.csv")
df.head(10)

,Fecha,Dia,Num_Day,Num_Month,Month,Year,Hora,Persona,Mensaje,clean_text
0,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,Capibaras Ironhack,‎Los mensajes y las llamadas están cifrados de...,mensaje llamada cifrado extremo extremo nadie ...
1,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,Alana Ironhack,‎Alana Ironhack creó el grupo “Capibaras Ironh...,aln ironhack creer grupo capibara ironhack
2,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,Capibaras Ironhack,‎Alana Ironhack te añadió.,aln ironhack t anadio
3,30/11/2023,Jueves,30,11,Nov,2023,19:12:56,Alana Ironhack,Hola,holar
4,30/11/2023,Jueves,30,11,Nov,2023,19:13:12,Alana Ironhack,Les paso las fotos de ayer prepárense xq es un...,l pasar foto ayer preparense ser bombardeo
5,30/11/2023,Jueves,30,11,Nov,2023,20:22:19,~ Teresa,Es un bombardeo que nos ahorra prácticamente u...,ser bombardeo ahorrar practicamente laboratorio
6,30/11/2023,Jueves,30,11,Nov,2023,20:22:38,Alana Ironhack,Jajaja,jajaja
7,30/11/2023,Jueves,30,11,Nov,2023,20:32:18,Sil🐨,Muchas gracias!,gracia
8,01/12/2023,Viernes,1,12,Dic,2023,17:10:16,~ Teresa,Fan del icono del grupo,fan icono grupo
9,01/12/2023,Viernes,1,12,Dic,2023,17:11:26,~ Teresa,Entregue el lab sin el último ejercicio del fa...,entregar laboratorio ultimo ejercicio facetgri...


In [4]:
def dividir_y_agrupar_por_tiempo(df, intervalo_tiempo):
    # Convierte la columna de fechas a tipo datetime si no lo está
    df['Fecha'] = pd.to_datetime(df['Fecha'])
    
    # Agrupa el DataFrame por el intervalo de tiempo deseado y la columna de conversación
    grupos = df.groupby([pd.Grouper(key='Fecha', freq=intervalo_tiempo), 'Persona'])['clean_text'].apply(list)
    
    # Devuelve el DataFrame original junto con la agrupación por conversación
    return grupos


# intervalo_tiempo puede ser 'D' para días, 'H' para horas, 'M' para minutos, etc.

#df, conversaciones_agrupadas = dividir_y_agrupar_por_tiempo(df, 'D')
a = dividir_y_agrupar_por_tiempo(df, '5H')
#pd.set_option("max_colwidth", None)
#a.to_frame()
a = pd.DataFrame(a)
a
# Ahora tienes tu DataFrame original df y un objeto conversaciones_agrupadas que contiene las conversaciones agrupadas por día
# Puedes acceder a las conversaciones agrupadas de la siguiente manera:
#df_final = conversaciones_agrupadas.reset_index()
#df_final

/var/folders/1m/9tdt_k994v175ff72qvvlr7r0000gn/T/ipykernel_27616/1121460803.py:3: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Fecha'] = pd.to_datetime(df['Fecha'])


clean_text
Fecha               Persona                                                              
2023-11-30 00:00:00 Alana Ironhack      [aln ironhack creer grupo capibara ironhack, h...
                    Capibaras Ironhack  [mensaje llamada cifrado extremo extremo nadie...
                    Sil🐨                                                         [gracia]
                    ~ Teresa            [ser bombardeo ahorrar practicamente laboratorio]
2023-11-30 20:00:00 Alana Ironhack      [hola l habiar encantar, hoy l termino haber c...
...                                                                                   ...
2024-02-15 22:00:00 Sil🐨                [capis vai, creer remonte, jajajjajar, ahora r...
                    ~ Teresa            [holiii, tener ser john, creer beer tasting se...
2024-02-16 23:00:00 Alana Ironhack      [holiii, l ir leer, l pasar l octavio victor, ...
                    Sil🐨                [jajajajajar, ay madre, pasar, l haber divir l...
                    ~ Teresa            [cappiiiis, anoche deje ordenador nuevo, concl...

[151 rows x 1 columns]

In [166]:
# Exportamos los datos limpios en un archivo csv
a.to_csv('../data/clear5H.csv', header=True, index=True)

In [167]:
# Ahora, definimos una función para analizar el sentimiento de un texto usando TextBlob
def analizar_sentimiento(texto):
    analysis = TextBlob(str(texto))
    polarity = analysis.sentiment.polarity
    if polarity > 0:
        return 'positivo'
    elif polarity < 0:
        return 'negativo'
    else:
        return 'neutro'

# Aplicamos la función a cada fila de la columna 'text_clean' y creamos una nueva columna 'sentimiento'
a['sentimiento'] = a['clean_text'].apply(analizar_sentimiento)
a['polarity']
a
# Ahora el DataFrame tiene una nueva columna llamada 'sentimiento' que contiene el resultado del análisis de sentimiento


clean_text  \
Fecha               Persona                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
2023-11-30 00:00:00 Alana Ironhack                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               [aln ironhack creer grupo capibara ironhack, holar, l pasar foto ayer preparense ser bombardeo, jajaja]   
                    Capibaras Ironhack                                                                                                                                                                                                                                                                                                                                                                                                                                                                             [mensaje llamada cifrado extremo extremo nadie chat siquiera whatsapp poder leer l escucharlo, aln ironhack t anadio]   
                    Sil🐨                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        [gracia]   
                    ~ Teresa                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           [ser bombardeo ahorrar practicamente laboratorio]   
2023-11-30 20:00:00 Alana Ironhack                                                                                                                                                                                                                                      [hola l habiar encantar, hoy l termino haber cosa saliar, bueno rosella haber reconocer buen trabajo nena, ahora ser capibaro enferma, cierto final si quedar mann si morir gripe jajajar, ahora frio mundir igual, si si trabajo ir caer, ser si ir morir ahi frio asi abandonar david ira amigo asi mal jajaj, si ademar asi dijera cuanta gana tener ir pues]   
...                                                                                                                                                                                                                                                                       

In [168]:

# Puedes guardar este DataFrame modificado en un nuevo archivo CSV si lo deseas
a.to_csv('../data/conversacion_5H_sentimiento.csv')